# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# import matplotlib_inline
# import os
import json
# import matplotlib.pyplot as plt
# import numpy as np
# import time
# from scipy.stats import linregress


# Import citipy to determine the cities based on latitude and longitude
# from citipy import citipy


# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", index_col=0)

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,dikson,73.5069,80.5464,-0.83,98,100,10.40,RU,1670623689
1,hami,42.8000,93.4500,23.49,44,95,6.87,CN,1670623690
2,kavieng,-2.5744,150.7967,82.89,75,100,8.95,PG,1670623690
3,hermanus,-34.4187,19.2345,67.80,85,0,3.29,ZA,1670623690
4,provideniya,64.3833,-173.3000,21.87,85,92,8.21,RU,1670623691


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
humidity_map= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    frame_width= 800,
    frame_height= 600,
    size= "Humidity",
    scale= 2,
    color= "Country",
    hover_cols=["Max Temp", "City"]
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Max Temp,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp= city_data_df.loc[(city_data_df['Max Temp']<=40)] #& (city_data_df['Max Temp']>=0)]
ideal_hum= ideal_temp.loc[(ideal_temp['Humidity'] <= 75)]
ideal_city= ideal_hum.loc[(ideal_hum['Wind Speed']<=7)]


# Drop any rows with null values
ideal_city= ideal_city.dropna()

# Display sample data
ideal_city.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
1,hami,42.8000,93.4500,23.49,44,95,6.87,CN,1670623690
11,albany,42.6001,-73.9662,33.35,53,0,6.29,US,1670623655
60,winnemucca,40.9730,-117.7357,39.25,48,100,4.61,US,1670623724
78,pavlodar,52.3000,76.9500,-18.60,69,0,6.71,KZ,1670623731
95,anadyr,64.7500,177.4833,5.50,71,0,6.71,RU,1670623747


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_city.reset_index(drop=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,hami,42.8000,93.4500,23.49,44,95,6.87,CN,1670623690,
1,albany,42.6001,-73.9662,33.35,53,0,6.29,US,1670623655,
2,winnemucca,40.9730,-117.7357,39.25,48,100,4.61,US,1670623724,
3,pavlodar,52.3000,76.9500,-18.60,69,0,6.71,KZ,1670623731,
4,anadyr,64.7500,177.4833,5.50,71,0,6.71,RU,1670623747,
5,jinchang,38.4953,102.1739,18.86,63,91,5.19,CN,1670623813,
6,gravdal,68.1214,13.5031,15.69,73,0,5.75,NO,1670623835,
7,fort nelson,58.8053,-122.7002,-0.06,65,92,1.99,CA,1670623838,
8,zhuanghe,39.7008,122.9911,32.68,45,100,4.14,CN,1670623850,
9,kristiansund,63.1115,7.7320,23.65,74,0,4.61,NO,1670623872,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,

    "limit": 20 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat= row["Lat"]
    lng= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    response = response.json()
  
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
  

# Display sample data
hotel_df

Starting hotel search
hami - nearest hotel: Хами
albany - nearest hotel: No hotel found
winnemucca - nearest hotel: The Overland Hotel
pavlodar - nearest hotel: Север
anadyr - nearest hotel: Гостевой дом
jinchang - nearest hotel: No hotel found
gravdal - nearest hotel: Scandic Leknes Lofoten
fort nelson - nearest hotel: Fort Nelson Hotel
zhuanghe - nearest hotel: 非繁城品酒店
kristiansund - nearest hotel: Comfort Hotel Fosna
leh - nearest hotel: Kang Lha Chen
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
sitka - nearest hotel: Aspen Suites Hotel
neyshabur - nearest hotel: هتل امیران
seoul - nearest hotel: 뉴국제호텔
dandong - nearest hotel: Lifes
khani - nearest hotel: No hotel found
gongzhuling - nearest hotel: 7天优品酒店
svetlaya - nearest hotel: No hotel found
lewiston - nearest hotel: Hampton
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
east millcreek - nearest hotel: Home2 Suites by Hilton Salt Lake City - East
semnan - nearest hotel: مرکز آموزشی رفاهی فرهنگیان - خانه معلم


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,hami,42.8000,93.4500,23.49,44,95,6.87,CN,1670623690,Хами
1,albany,42.6001,-73.9662,33.35,53,0,6.29,US,1670623655,No hotel found
2,winnemucca,40.9730,-117.7357,39.25,48,100,4.61,US,1670623724,The Overland Hotel
3,pavlodar,52.3000,76.9500,-18.60,69,0,6.71,KZ,1670623731,Север
4,anadyr,64.7500,177.4833,5.50,71,0,6.71,RU,1670623747,Гостевой дом
5,jinchang,38.4953,102.1739,18.86,63,91,5.19,CN,1670623813,No hotel found
6,gravdal,68.1214,13.5031,15.69,73,0,5.75,NO,1670623835,Scandic Leknes Lofoten
7,fort nelson,58.8053,-122.7002,-0.06,65,92,1.99,CA,1670623838,Fort Nelson Hotel
8,zhuanghe,39.7008,122.9911,32.68,45,100,4.14,CN,1670623850,非繁城品酒店
9,kristiansund,63.1115,7.7320,23.65,74,0,4.61,NO,1670623872,Comfort Hotel Fosna


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    frame_width= 800,
    frame_height= 600,
    size= "Humidity",
    scale= 2,
    color= "Country",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)